In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
import cv2
from sklearn.model_selection import train_test_split
import pickle
import os
import pandas as pd
import random
from keras.preprocessing.image import ImageDataGenerator

################# Parameters #####################

# folder with all the class folders
path = "D:\Machine Learning Projects\CNN notebooks\myData"
# file with all names of classes
labelFile = 'D:\Machine Learning Projects\CNN notebooks\labels.csv'
batch_size_val = 8  # how many to process together
steps_per_epoch_val = 2000
epochs_val = 20
imageDimesions = (256, 256, 3)
testRatio = 0.2    # if 1000 images split will 200 for testing
validationRatio = 0.2  # if 1000 images 20% of remaining 800 will be 160 for validation
###################################################


# Importing of the Images
count = 0
images = []
classNo = []
myList = os.listdir(path)
print("Total Classes Detected:", len(myList))
noOfClasses = len(myList)
print("Importing Classes.....")
for x in range(0, len(myList)):
    myPicList = os.listdir(path+"/"+str(count))
    for y in myPicList:
        curImg = cv2.imread(path+"/"+str(count)+"/"+y)
        curImg = cv2.resize(curImg, (256, 256), interpolation=cv2.INTER_AREA)
        images.append(curImg)
        classNo.append(count)
    print(count, end=" ")
    count += 1
print(" ")
images = np.array(images)
classNo = np.array(classNo)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(
    images, classNo, test_size=testRatio)
X_train, X_validation, y_train, y_validation = train_test_split(
    X_train, y_train, test_size=validationRatio)

# X_train = ARRAY OF IMAGES TO TRAIN
# y_train = CORRESPONDING CLASS ID

# TO CHECK IF NUMBER OF IMAGES MATCHES TO NUMBER OF LABELS FOR EACH DATA SET
print("Data Shapes")
print("Train", end="")
print(X_train.shape, y_train.shape)
print("Validation", end="")
print(X_validation.shape, y_validation.shape)
print("Test", end="")
print(X_test.shape, y_test.shape)
assert (X_train.shape[0] == y_train.shape[0]
        ), "The number of images in not equal to the number of lables in training set"
assert (X_validation.shape[0] == y_validation.shape[0]
        ), "The number of images in not equal to the number of lables in validation set"
assert (X_test.shape[0] == y_test.shape[0]
        ), "The number of images in not equal to the number of lables in test set"
assert (X_train.shape[1:] == (imageDimesions)
        ), " The dimesions of the Training images are wrong "
assert (X_validation.shape[1:] == (imageDimesions)
        ), " The dimesionas of the Validation images are wrong "
assert (X_test.shape[1:] == (imageDimesions)
        ), " The dimesionas of the Test images are wrong"


# READ CSV FILE
data = pd.read_csv(labelFile)
print("data shape ", data.shape, type(data))


# PREPROCESSING THE IMAGES


def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img


def equalize(img):
    img = cv2.equalizeHist(img)
    return img


def preprocessing(img):
    img = grayscale(img)     # CONVERT TO GRAYSCALE
    img = equalize(img)      # STANDARDIZE THE LIGHTING IN AN IMAGE
    img = img/255            # TO NORMALIZE VALUES BETWEEN 0 AND 1 INSTEAD OF 0 TO 255
    return img


# TO IRETATE AND PREPROCESS ALL IMAGES
X_train = np.array(list(map(preprocessing, X_train)))
X_validation = np.array(list(map(preprocessing, X_validation)))
X_test = np.array(list(map(preprocessing, X_test)))
# TO CHECK IF THE TRAINING IS DONE PROPERLY

# ADD A DEPTH OF 1
X_train = X_train.reshape(
    X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_validation = X_validation.reshape(
    X_validation.shape[0], X_validation.shape[1], X_validation.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)


# AUGMENTATAION OF IMAGES: TO MAKEIT MORE GENERIC
dataGen = ImageDataGenerator(width_shift_range=0.1,   # 0.1 = 10%     IF MORE THAN 1 E.G 10 THEN IT REFFERS TO NO. OF  PIXELS EG 10 PIXELS
                             height_shift_range=0.1,
                             zoom_range=0.2,  # 0.2 MEANS CAN GO FROM 0.8 TO 1.2
                             shear_range=0.1,  # MAGNITUDE OF SHEAR ANGLE
                             rotation_range=10,
                             )  # DEGREES
dataGen.fit(X_train)
# REQUESTING DATA GENRATOR TO GENERATE IMAGES  BATCH SIZE = NO. OF IMAGES CREAED EACH TIME ITS CALLED
batches = dataGen.flow(X_train, y_train, batch_size=20)
X_batch, y_batch = next(batches)





In [ ]:
classes = ['oaifa','khubaib','zeeshan']

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
cnn = models.Sequential([
        #cnn
        layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu',input_shape=(256,256,1)),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
        layers.MaxPooling2D((2,2)),

        

        #dense
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')    
    ])

cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

cnn.fit(X_batch, y_batch, epochs=10)

In [ ]:
cnn.evaluate(X_test,y_test)

### testing

In [ ]:
import cv2  
import numpy as np
def grayscale(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img


def equalize(img):
    img = cv2.equalizeHist(img)
    return img


def preprocessing(img):
    img = grayscale(img)
    img = equalize(img)
    img = img/255
    return img


def getCalssName(classNo):
    if classNo == 0:
        return 'ozaifa'
    elif classNo == 1:
        return 'Khubaib'
    elif classNo == 2:
        return 'zeeshan'


imgOrignal = cv2.imread(
    r"myData\1\WhatsApp Image 2023-03-29 at 4.58.12 PM.jpeg")

# PROCESS IMAGE
img = np.asarray(imgOrignal)
img = cv2.resize(img, (256, 256), interpolation=cv2.INTER_AREA)
img = preprocessing(img)
cv2.imshow("Processed Image", img)
img = img.reshape(1, 256, 256, 1)

In [ ]:
y_pred = cnn.predict(img)
print(classes[np.argmax(y_pred)],np.amax(y_pred))
plt.imshow(cv2.imread(r"myData\1\WhatsApp Image 2023-03-29 at 4.58.12 PM.jpeg"))

### Saving the model

In [ ]:

cnn.save("Face_recognition.h5")

#### Using .h5 model

In [ ]:
from tensorflow.keras.models import load_model
model = load_model("Face_recognition.h5")
y_pred=model.predict(img)
print(classes[np.argmax(y_pred)],np.amax(y_pred))
plt.imshow(cv2.imread(r"myData\1\WhatsApp Image 2023-03-29 at 4.58.12 PM.jpeg"))